### 1. Import Dependencies

In [1]:
#Modules
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gdp
import os
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(3)
#np.random.seed(0)

### 2. Define GAN Functions

In [ ]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob, D_logit

def xavier_init(size):
    input_dim = size[0]
    xavier_variance = 1. / tf.sqrt(input_dim/2.)
    return tf.random_normal(shape=size, stddev=xavier_variance)

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gdp.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis("off")
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect("equal")
        plt.imshow(sample.reshape(28, 28), cmap="Greys_r")
    return fig

def sample_Z(m, n):
    a = np.random.uniform(-1.,1., size=[m,n])
    print(a)
    return a

### 3. Init Generator

In [ ]:
# Function call
Z = tf.placeholder(tf.float32, shape = [None, 100], name = "Z")
X = tf.placeholder(tf.float32, shape = [None, 784], name = "X")

# Weight and bias matrices for generator
G_W1 = tf.Variable(xavier_init([100, 128]), name = "G_W1") 
G_b1 = tf.Variable(tf.zeros(shape =[128]), name = "G_b1")
G_W2 = tf.Variable(xavier_init([128, 784]), name = "G_W2")
G_b2 = tf.Variable(tf.zeros(shape =[784]), name = "G_b2")
theta_G = [G_W1, G_W2, G_b1, G_b2]

# Weight and bias matrices for discriminator
D_W1 = tf.Variable(xavier_init([784, 128]), name = "D_W1")
D_b1 = tf.Variable(tf.zeros(shape =[128]), name = "D_b1")
D_W2 = tf.Variable(xavier_init([128, 1]), name = "D_W2")
D_b2 = tf.Variable(tf.zeros(shape =[1]), name = "D_b2")
theta_D = [D_W1, D_W2, D_b1, D_b2]

G_sample = generator(Z)

D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log1p(-D_fake)) #Replace with tf.log1p(-D_fake): Tends to be more numerically stable
G_loss = -tf.reduce_mean(tf.log(D_fake))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

### 4. Test on MNIST

In [ ]:
batch_size = 128
Z_dim = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

mnist = input_data.read_data_sets("MNIST/", one_hot = True)

if not os.path.exists("seed3/"):
    os.makedirs("seed3/")

i = 0
step = 1000000

for itr in range(step):
    if itr % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig("seed3/{}.png".format(str(i).zfill(3)), bbox_inches="tight")
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(batch_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict ={X: X_mb, Z: sample_Z(batch_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict ={Z: sample_Z(batch_size, Z_dim)})

    if itr % 1000 == 0:
        print("Iter: {}".format(itr))
        print("D_loss: {:.4}".format(D_loss_curr))
        print("G_loss: {:.4}".format(G_loss_curr))
        print()

In [ ]:
"""
Potential problems:

1. G_loss and D_loss error messaged: "Masked values being converted to NAN"
    #1: Try fix in line 67 to make more numerically stable
    #2: Try with another random seed
    #3: May be experiencing "excessive mode dropping"
        - Meaning: The generator is mapping to some common output that 
        happens to fool the discriminator with an extremely high probability
        - Very common
        - Solution: add a small epsilon to the log losses (e.g. 1e-05) so 
        that regardless of how good/bad the quality of G(z) or D(x) are, 
        you're within a stabl region of the log function
            - Note: Won't solve underlying issues, output will still be meaningless,
            but at least NAN stops

"""